In [1]:
%matplotlib inline

import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import math
import pickle
import pandas as pd
import scipy.io
import time
import h5py
import bz2
import multiprocessing as mp


from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

from numpy import linalg as LA
from scipy.spatial import Delaunay
from sklearn.neighbors import NearestNeighbors, KDTree

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import matthews_corrcoef

#sys.path.insert(0, "../")
from info3d import *
from nn_matchers import *

# Extracting the point collection and the descriptors

In [3]:
with open('point_collection/new_contiguous_point_collection.pickle','rb') as f: 
    new_contiguous_point_collection = pickle.load(f)
    
with open('descriptors/new_complete_res5_4by5_descriptors.pickle','rb') as f:
    descriptors = pickle.load(f)
    
with open('descriptors/new_complete_RANSAC_res5_4by5_descriptors.pickle','rb') as f:
    ransac_descriptors = pickle.load(f)

# Raw spaces (validation)

In [ ]:
descriptors = descriptors

with open('sample_points.pickle','rb') as f:
    sample_points = pickle.load(f)

for radius in np.arange(0.5,2.1,0.25):
    
    t0 = time.time()
    t1 = time.time()

    
    partial_scores_pool = []
    
    print("radius =",radius)
    
    for obj_, object_name, original_vertex in sample_points:
        
        new_partial_pointcloud = []
        new_vX = []
        new_vZ = []
        
        try:

            object_, pointCloud, tri_ = new_contiguous_point_collection[int(obj_)]
            
            ransac_nbrs = NearestNeighbors(n_neighbors=min(20000,len(pointCloud)), algorithm='kd_tree').fit(ransac_pointCloud[:,:3])
            
            dist_, ind_ = ransac_nbrs.kneighbors([original_vertex[:3]])
            pointCloud =  ransac_pointCloud[ind_[0,np.where(dist_[0,:]<=radius)[0]]]
        except:
            print("Can't get partial samples for",obj_meta[0])
            continue
            
        #if len(gen_planes) == 0: continue
        if len(pointCloud) == 0: continue

        local_keypoint_matches = []

        try:
            obj_meta, diff_ratios, diff_indexs, diff_scores, local_keypoint_matches = get_score_kdtree_lean(
                [obj_, object_name, original_vertex], 
                pointCloud, 
                descriptors
                #RANSAC = True,
                # old = True
            )
            
        except Exception as ex:
            print(ex)
            print("skipped",object_name)
            continue
        
        if len(local_keypoint_matches) == 0: 
            print("skipped",object_name)
            continue

        partial_scores_pool.append([
            [obj_, object_name, original_vertex], 
            diff_ratios,
            diff_indexs,
            diff_scores,
            local_keypoint_matches
        ])

        if len(partial_scores_pool) % 33 == 1:
            
            partial_errors_pool = NN_matcher(partial_scores_pool)
            print(radius,"Error Rate:",)
            
            print("  ",radius,": Done with {}, in {:.3f} seconds. Error rate {:.3f}".format(
                len(partial_scores_pool),
                time.time()-t1,
                np.sum(partial_errors_pool[:,1])/len(partial_scores_pool)
            ))
                        
            with bz2.BZ2File('testing_results/partial/{}_partial_scores.pickle.bz2'.format(radius), 'w') as bz2_f:
                pickle.dump(partial_scores_pool, bz2_f)
                
            t1  = time.time()

    print(radius," Total Time to match {:.3f} seconds.".format(time.time()-t0))
    

    #print(len(partial_lengths))
       

In [ ]:
descriptors = ransac_descriptors

with open('sample_points.pickle','rb') as f:
    sample_points = pickle.load(f)

for radius in np.arange(2.5,3.1,0.25):
    
    t0 = time.time()
    t1 = time.time()

    
    partial_scores_pool = []
    
    print("radius =",radius)
    
    for obj_, object_name, original_vertex in sample_points:
        
        new_partial_pointcloud = []
        new_vX = []
        new_vZ = []
        
        try:
            trial = np.random.randint(5)
            
            with open("../spatial-privacy-pointnetvlad/ransac_pc/ransac_point_collection_{}.pickle".format(trial),'rb') as f:
                ransac_trial_point_collection = pickle.load(f)

            object_, ransac_pointCloud, tri_ = ransac_trial_point_collection[int(obj_)]
            
            ransac_nbrs = NearestNeighbors(n_neighbors=min(20000,len(ransac_pointCloud)), algorithm='kd_tree').fit(ransac_pointCloud[:,:3])
            
            dist_, ind_ = ransac_nbrs.kneighbors([original_vertex[:3]])
            pointCloud =  ransac_pointCloud[ind_[0,np.where(dist_[0,:]<=radius)[0]]]
        except:
            print("Can't get ransac samples for",trial,obj_meta[0],dist_.shape,ind_.shape)
            continue
            
        #if len(gen_planes) == 0: continue
        if len(pointCloud) == 0: continue

        local_keypoint_matches = []

        try:
            obj_meta, diff_ratios, diff_indexs, diff_scores, local_keypoint_matches = get_score_kdtree_lean(
                [obj_, object_name, original_vertex], 
                pointCloud, 
                descriptors
                #RANSAC = True,
                # old = True
            )
            
        except Exception as ex:
            print(ex)
            print("skipped",object_name)
            continue
        
        if len(local_keypoint_matches) == 0: 
            print("skipped",object_name)
            continue

        partial_scores_pool.append([
            [obj_, object_name, original_vertex], 
            diff_ratios,
            diff_indexs,
            diff_scores,
            local_keypoint_matches
        ])

        if len(partial_scores_pool) % 33 == 1:
            
            partial_errors_pool = NN_matcher(partial_scores_pool)
            print(radius,"Error Rate:",)
            
            print("  ",radius,": Done with {}, in {:.3f} seconds. Error rate {:.3f}".format(
                len(partial_scores_pool),
                time.time()-t1,
                np.sum(partial_errors_pool[:,1])/len(partial_scores_pool)
            ))
                        
            with bz2.BZ2File('parallel_results/{}_partial_scores_parallel_sample_points.pickle.bz2'.format(radius), 'w') as bz2_f:
                pickle.dump(partial_scores_pool, bz2_f)
                
            t1  = time.time()

    print(radius," Total Time to match {:.3f} seconds.".format(time.time()-t0))
    

    #print(len(partial_lengths))
       